**МОДЕЛЬ ПРОГНОЗИРОВАНИЯ ВОЗВРАТА КРЕДИТОВ**

Импортируем все необходимые библиотеки и функции

In [152]:
import pandas as pd, numpy as np,collections as cl, matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.utils.multiclass import unique_labels

Для начала считаем данные из файла

In [153]:
data = pd.read_csv('../input/hmeq.csv')

В выборке есть пустые значения. Для простоты удалим все строки, в которых есть пустые значения.

In [154]:
data.dropna(how='any', inplace=True)

Далее применим базовые процедуры предобработки данных, такие как нормализация и дискретизация. Нормализовывать мы будем все числовые признаки по Z-показателю (масштабирование данных на основе среднего значения признака и стандартного отклонения). После нормализации разобъем числовые признаки на группы равной ширины при помощи функции Cut. 

In [155]:
for column in data:
    if column not in ['BAD','REASON','JOB']:         
        mean=data[column].mean()       
        std=data[column].std()       
        data[column]=[(x-mean)/std for x in data[column]]
        data[column]=pd.cut(data[column],20)    

Используем LabelEncoder, чтобы заменить группу её номером.

In [156]:
le = LabelEncoder()
for column in data.columns:
    encc = le.fit(data[column].astype(str))  
    tr = le.transform(data[column].astype(str))
    data[column]=tr

Запишем целевой признак в отдельную переменную и удалим соотвествующий столбец из первоначальной выборки

In [157]:
data1 = data.copy()
bad=data['BAD']
del data['BAD']

Разделим выборку на тренировочную и тестирующую (33% от изначального размера выборки)

In [158]:
x_train, x_test, y_train, y_test = train_test_split(data, bad, test_size=0.33, random_state=42)   

Используем 3 классификатора для оценки работы модели. Первый из них - это DesicionTreeClassifier. Для оценки работы классификаторов мы используем accuracy, precision, recall и f1-score.

In [159]:
clrTree = DecisionTreeClassifier(max_depth = 3)
clrTree = clrTree.fit(x_train, y_train)
pred = list(clrTree.predict(x_test))   
precision=precision_score(y_test,pred)
recall=recall_score(y_test,pred)
f1 = f1_score(y_test,pred)
print("Accuracy decisiontree = {0:.5f}".format(clrTree.score(x_test,y_test)))
print("Precision= {0:.5f}\nRecall = {1:.5f}\nF1={2:.5f}\n".format(precision,recall,f1))

Accuracy decisiontree = 0.93699
Precision= 0.94118
Recall = 0.32000
F1=0.47761



Аналогичный код и для классификатора KNeighborsClassifier

In [160]:
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(x_train, y_train) 
pred = list(neigh.predict(x_test))  
precision=precision_score(y_test,pred)
recall=recall_score(y_test,pred)
f1 = f1_score(y_test,pred)
print("Accuracy KNeighborsClassifier = {0:.5f}".format(neigh.score(x_test,y_test)))
print("Precision= {0:.5f}\nRecall = {1:.5f}\nF1={2:.5f}\n".format(precision,recall,f1))

Accuracy KNeighborsClassifier = 0.92619
Precision= 0.87500
Recall = 0.21000
F1=0.33871



Третьим классификатором возьмем RandomForestClassifier

In [161]:
rf = RandomForestClassifier(max_depth=10)
rf.fit(x_train, y_train)
pred = list(rf.predict(x_test))  
precision=precision_score(y_test,pred)
recall=recall_score(y_test,pred)
f1 = f1_score(y_test,pred)
print("Accuracy RandomForestClassifier = {0:.5f}".format(rf.score(x_test,y_test)))
print("Precision= {0:.5f}\nRecall = {1:.5f}\nF1={2:.5f}\n".format(precision,recall,f1))

Accuracy RandomForestClassifier = 0.94149
Precision= 0.97297
Recall = 0.36000
F1=0.52555



Все три классификатора дают довольно неплохие результаты по accuracy, однако recall оставляет желать лучшего. В частности, это можно объяснить тем, что значения целевой переменной не являются сбалансированными. Значения, соотвествующие возврату кредита, составляют около 80% от всех значений целевой переменной. Для улучшения recall прибегнем к одной фишечке. В данной модели 0 - кредит будет возвращен, 1 - кредит не вернут. Заменим нули на единицы, а единицы на нули. В таком случае 0 будет означать, что кредит не вернут, а 1, соотвественно, что кредит вернут. Также стоит обратить внимание, что среди трех классификаторов наилучший результат дает RandomForest.

In [162]:
zero_index=data1[data1["BAD"]==0].index
one_index=data1[data1["BAD"]==1].index
data1['BAD'].loc[zero_index] = 1
data1['BAD'].loc[one_index] = 0
bad=data1['BAD']
del data1['BAD']

Повторим тот же эксперимент с теми же классификаторами.

In [163]:
x_train, x_test, y_train, y_test = train_test_split(data1, bad, test_size=0.33, random_state=42)   

clrTree = DecisionTreeClassifier(max_depth = 3)
clrTree = clrTree.fit(x_train, y_train)
pred = list(clrTree.predict(x_test))   
precision=precision_score(y_test,pred)
recall=recall_score(y_test,pred)
f1 = f1_score(y_test,pred)
print("Accuracy decisiontree = {0:.5f}".format(clrTree.score(x_test,y_test)))
print("Precision= {0:.5f}\nRecall = {1:.5f}\nF1={2:.5f}\n".format(precision,recall,f1))

neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(x_train, y_train) 
pred = list(neigh.predict(x_test))  
precision=precision_score(y_test,pred)
recall=recall_score(y_test,pred)
f1 = f1_score(y_test,pred)
print("Accuracy KNeighborsClassifier = {0:.5f}".format(neigh.score(x_test,y_test)))
print("Precision= {0:.5f}\nRecall = {1:.5f}\nF1={2:.5f}\n".format(precision,recall,f1))

rf = RandomForestClassifier(max_depth=10)
rf.fit(x_train, y_train)
pred = list(rf.predict(x_test))  
precision=precision_score(y_test,pred)
recall=recall_score(y_test,pred)
f1 = f1_score(y_test,pred)
print("Accuracy RandomForestClassifier = {0:.5f}".format(rf.score(x_test,y_test)))
print("Precision= {0:.5f}\nRecall = {1:.5f}\nF1={2:.5f}\n".format(precision,recall,f1))

Accuracy decisiontree = 0.93699
Precision= 0.93686
Recall = 0.99802
F1=0.96648

Accuracy KNeighborsClassifier = 0.92619
Precision= 0.92732
Recall = 0.99703
F1=0.96092

Accuracy RandomForestClassifier = 0.94329
Precision= 0.94299
Recall = 0.99802
F1=0.96973



Невооруженным глазом заметно, что recall заметно вырос, что повлияло и на f1-score. Примененная фишечка сделала модели прогнозирования возврата кредита намного лучше. Если сравнить классификаторы между собой, то снова лучший результат выдал RandomForest, однако в этот раз f1-score всех трех классификаторов не так сильно отличаются, как в предыдущем эксперименте.